In [ ]:
import face_recognition
import tensorflow as tf 
import cv2
from PIL import Image
import glob
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from keras.models import Sequential
from tensorflow.keras.utils import plot_model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, BatchNormalization, Activation


## Video to frames 


import cv2
# Playing video from file:
cap = cv2.VideoCapture("E:\\Final_year Project\\Training Videos\\FEAR\\VID_20220928_140332.mp4")
#fourcc = cv2.VideoWriter_fourcc(*'XVID')

try:
    if not os.path.exists('data'):
        os.makedirs('data')
except OSError:
    print ('Error: Creating directory of data')

currentFrame = 0
while(True):
    # Capture frame-by-frame
    try:
        ret, frame = cap.read()

        # Saves image of the current frame in jpg file
        #name = './data/frame' + str(currentFrame) + '.jpg'
        print ('Creating...' , '.\\Training Dataset\\FEAR\\frame%d.jpg' % currentFrame)
        cv2.imwrite('E:\\Final_year Project\\Training Dataset\\FEAR\\FEAR3(%d).jpg' % currentFrame, frame)   

        # To stop duplicate images
        currentFrame += 1
    except:
        break
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()


## Basic Face detection

cap = cv2.VideoCapture("E:\\Final_year Project\\Training Videos\\HAPPY\\VID_20220911_111728.mp4")

face_locations = []
while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    
    frame = np.array(frame, dtype=np.uint8)

    # Convert the image from BGR color (which OpenCV uses) to RGB   
    # color (which face_recognition uses)
    rgb_frame = frame[:, :, ::-1]
    
    # Find all the faces in the current frame of video
    face_locations = face_recognition.face_locations(rgb_frame)
    for top, right, bottom, left in face_locations:
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0,  
        255), 2)
    #Display the resulting image
    cv2.imshow('Video', frame)
    

    # Wait for Enter key to stop
    if cv2.waitKey(25) == 13:
        break
        

## Pre-processing Data

Datadir = "E:\\Final_year Project\\Training Dataset"  ##training dataset
Classes = ["ANGRY", "DISGUST", "FEAR", "HAPPY", "NEUTRAL", "SAD","SURPRISED"] #list of classes
for category in Classes:
    path = os.path.join(Datadir, category) ##//
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img))
        plt.imshow(cv2.cvtColor(img_array,cv2.COLOR_BGR2RGB))
        plt.show()
        break
    break
img_size = 224 ##ImageNet => 224 x 224
new_array = cv2.resize(img_array, (img_size,img_size))
plt.imshow(cv2.cvtColor(new_array,cv2.COLOR_BGR2RGB))
plt.show()

training_data = [] ##data

def create_training_Data():
    for category in Classes:
        path = os.path.join(Datadir, category)
        class_num = Classes.index(category) ##0, 1 ##label
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img))
                new_array = cv2.resize(img_array, (img_size,img_size))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass

import random

random.shuffle(training_data)
x = [] ##data/feature
y = [] ##label

for features,label in training_data:
    x.append(features)
    y.append(label)

x = np.array(x).reshape(-1, img_size, img_size, 3) ## converting it to 4 dimension

#normalize the data
x = x/255.0 ; ##normalizing it ## sckit learn

In [ ]:
# Deep learning model for training



#Initialize image data generator with rescaling
train_data_gen=ImageDataGenerator(rescale=1./255)
validation_data_gen=ImageDataGenerator(rescale=1./255)

train_generator=train_data_gen.flow_from_directory(
              'E:\\Final_year Project\\Training Dataset',
              target_size=(48, 48),
              batch_size=64,
              color_mode="grayscale",
              class_mode='categorical')

validation_generator=validation_data_gen.flow_from_directory(
            'E:\\Final_year Project\\Testing Dataset',
             target_size=(48, 48),
             batch_size=64,
             color_mode="grayscale",
             class_mode='categorical')

#create AlexNETmodel structure
emotion_model=Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=(48,48,1)))

emotion_model.add(Conv2D(64,kernel_size=(3, 3), activation='relu',padding = 'Same',))
emotion_model.add(MaxPooling2D(pool_size = (2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, (3, 3), activation='relu',padding = 'Same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))

emotion_model.add(Conv2D(256, (3, 3), activation='relu',padding = 'Same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.40))

emotion_model.add(Conv2D(512, (3, 3), activation='relu',padding = 'Same'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'], experimental_run_tf_function=False)

#Train the neural network/model
emotion_model_info=emotion_model.fit(
     train_generator,
     steps_per_epoch=18670 // 64,
     epochs = 15,
     validation_data=validation_generator,
     validation_steps=11200 // 64)

#save model structure in json file
model_json=emotion_model.to_json()
with open("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\emotion_model_new.json", "w") as json_file:
    json_file.write(model_json)
    
#save trained model weight in .h5 file
emotion_model.save_weights('C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\emotion_model_new.h5')



emotion_model.summary()

checkpoint=ModelCheckpoint("C:\\Users\\Lenovobhavya\\Documents\\Project_PBL\\data\\emotion_model_new.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode= 'max')
early_stopping=EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=3,
                            verbose=1,
                            restore_best_weights=True)
reduce_learningrate = ReduceLROnPlateau(monitor= 'val_loss',
                                factor = 0.2,
                                patience=3,
                                 verbose = 1,
                                min_delta=0.0001)
callbacks_list = [early_stopping, checkpoint, reduce_learningrate]

history = emotion_model.fit(
     train_generator,
     steps_per_epoch=7779 // 64,
     epochs = 15,
     validation_data=validation_generator,
     validation_steps=7519 // 64,
     callbacks = [early_stopping, checkpoint, reduce_learningrate])
print("Done!")

plt.figure (figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle( 'Optimizer:Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label= 'Training Loss')
plt.plot(history.history['val_loss'], label= 'Validation Loss')
plt.legend (loc='upper right')
plt.subplot (1, 2, 2)
plt.ylabel('Accuracy')
plt.plot(history.history['accuracy'], label= 'Training Accuracy')
plt.plot(history.history['val_accuracy'], label= 'Validation Accuracy')
plt.legend (loc= 'lower right')
plt.show()